In [1]:
import tensorflow as tf

# Load in original data

In [2]:
data_fname = '../wnut_ner_evaluation/data/train_notypes'
xs, ys = [], []
with open(data_fname, 'r') as f:
    x, y = [], []
    for i, line in enumerate(f):
        split = line.split()
        if split:
            x.append(split[0])
            y.append(split[1])
        else: 
            xs.append(x)
            ys.append(y)
            x, y = [], []

data_fname = '../wnut_ner_evaluation/data/dev_notypes'
dev_xs, dev_ys = [], []
with open(data_fname, 'r') as f:
    x, y = [], []
    for i, line in enumerate(f):
        split = line.split()
        if split:
            x.append(split[0])
            y.append(split[1])
        else: 
            dev_xs.append(x)
            dev_ys.append(y)
            x, y = [], []

In [3]:
from preprocess import *
from vocab import Vocab
from evaluation import macro_f1

# Word Level Model

In [4]:
# all_words = [ word for x in xs for word in x ]
# all_labels = [l for y in ys for l in y ]
# xvocab = Vocab(all_words, min_count=0)
# yvocab = Vocab(all_labels, min_count=0)
# print xvocab.n

In [5]:
# all_xs = xs + dev_xs
# all_ys = ys + dev_ys

# all_xs = sequences_to_index(all_xs, xvocab)
# all_xs, all_ws, max_time = pad_sequences(all_xs, xvocab.ipad)
# all_ys = sequences_to_index(all_ys, yvocab)
# all_ys, _, _ = pad_sequences(all_ys, yvocab.ipad)

# train_xs, test_xs = all_xs[:len(xs)], all_xs[len(xs):]
# train_ws, test_ws = all_ws[:len(xs)], all_ws[len(xs):]
# train_ys, test_ys = all_ys[:len(xs)], all_ys[len(xs):]

# print max_time
# print len(train_xs), len(test_xs)

In [6]:
# print train_xs[0]
# print train_ws[0]
# print train_ys[0]

In [7]:
# from wordner import WordNER

In [8]:
# tf.reset_default_graph()
# session = tf.InteractiveSession()

# params = {
#     'learning_rate':.001,
#     'xvocab':xvocab,
#     'yvocab':yvocab,
#     'max_time':max_time,
#     'word_embed_size':50,
#     'hidden_size':100,
#     'dropout':.0
# }
# ner = WordNER(session, **params)

In [9]:
# ner.fit(zip(train_xs, train_ws), train_ys, batch_size=64, n_epoch=5)

In [10]:
# preds = ner.predict(zip(test_xs, test_ws))
# preds = preds.tolist()

In [11]:
# target_ys = depad_sequences(test_ys, yvocab.ipad)
# predictions = depad_sequences(preds, test_ws)

In [12]:
# flat_targets = [ yvocab.token(t) for y in target_ys for t in y ]
# flat_predictions = [ yvocab.token(p) for y in predictions for p in y ]
# stats = macro_f1(flat_targets, flat_predictions)

In [13]:
# print stats

# Character Level Model

In [14]:
all_chargrams = [ c for x in xs for g in chargrams(x) for c in g ]
xvocab = Vocab(all_chargrams, min_count=0)
yvocab = Vocab([l for y in ys for l in y ], min_count=0)
print xvocab.n

286687


In [15]:
all_xs = xs + dev_xs
all_ys = ys + dev_ys
print yvocab.v
print set([ y for ys in all_ys for y in ys])

all_xs, all_ws = sentences_to_chargrams(all_xs, xvocab)
all_xs, all_ws, max_time, max_chargrams = pad_chargrams(all_xs, all_ws, xvocab.ipad)
all_ys = sequences_to_indices(all_ys, yvocab)
all_ys, _, _ = pad_sequences(all_ys, yvocab.ipad)

train_xs, test_xs = all_xs[:len(xs)], all_xs[len(xs):]
train_ws, test_ws = all_ws[:len(xs)], all_ws[len(xs):]
train_ys, test_ys = all_ys[:len(xs)], all_ys[len(xs):]

print max_time, max_chargrams
print len(train_xs), len(test_xs)

5
set(['I', 'B', 'O'])
41 123
2394 1000


In [16]:
print set([ y for ys in all_ys for y in ys])

set([0, 1, 2, 3])


In [17]:
print yvocab._vocab2idx
print yvocab.v

{'I': 3, '<PAD>': 0, 'B': 2, 'O': 1, '<UNK>': 1}
5


In [18]:
#from charner import ChargramNER
from tfmodel import *

class ChargramNER(TFModel):
    def build_forward(self):
        # inputs
        self.x_input = tf.placeholder(tf.int32, [None, self.max_time, self.max_chargrams])
        self.x_weight = tf.placeholder(tf.float32, [None, self.max_time, self.max_chargrams])
        self.y_input = tf.placeholder(tf.int32, [None, self.max_time])
        self.dropout_keep = tf.placeholder(tf.float32)
        
        # embed and take weighted sum of character grams as word embedding
        self.chargram_vectors = tf.Variable(tf.random_uniform([self.xvocab.v, self.char_embed_size], 
                                                             -.1, .1, tf.float32))
        self.embedded_chargrams = tf.nn.embedding_lookup(self.chargram_vectors, 
                                                         self.x_input)
        tile_weights = tf.tile(tf.expand_dims(self.x_weight, [3]), [1,1,1,self.char_embed_size])
        self.embedded_words = tf.reduce_sum(tile_weights * self.embedded_chargrams, [2])
        
        # convert sequences to list of timesteps
        self.rnn_inputs = [ tf.squeeze(i, [1]) for i in tf.split(1, self.max_time, self.embedded_words) ]
        
        # bidirectional encoder
        fw_cell = tf.nn.rnn_cell.GRUCell(self.hidden_size)
        bw_cell = tf.nn.rnn_cell.GRUCell(self.hidden_size)
        outputs, _, _ = tf.nn.bidirectional_rnn(fw_cell, bw_cell,
                                                self.rnn_inputs,
                                                dtype=tf.float32)
    
        # hidden layer
#         self.Wh = tf.Variable(tf.random_uniform([self.char_embed_size, self.hidden_size], 
#                                               -.1, .1, tf.float32))
#         self.bh = tf.Variable(tf.zeros([self.hidden_size]))
#         hidden = tf.matmul(tf.reshape(self.embedded_words, [-1, self.char_embed_size]), self.Wh) + self.bh
#         hidden = tf.nn.relu(hidden)
        
        # softmax layer
        self.W = tf.Variable(tf.random_uniform([2*self.hidden_size, self.yvocab.v], 
                                              -.1, .1, tf.float32))
        self.b = tf.Variable(tf.zeros([self.yvocab.v]))
        
        self.logits = [ tf.matmul(output, self.W) + self.b for output in outputs ]     
        print len(self.logits), self.logits[0].get_shape()
        self.labels = [ tf.argmax(logit, 1) for logit in self.logits ]
        
    def build_loss(self):
        word_weights = tf.minimum(tf.reduce_sum(self.x_weight, [2]), 1.)
        # convert logits to lists
        #logits_list = [tf.squeeze(t, [1]) for t in tf.split(1, self.max_time, self.logits)]
        targets_list = [tf.squeeze(t, [1]) for t in tf.split(1, self.max_time, self.y_input)]
        weights_list = [tf.squeeze(t, [1]) for t in tf.split(1, self.max_time, word_weights)]
        seq_loss = tf.nn.seq2seq.sequence_loss_by_example(self.logits,
                                                          targets_list,
                                                          weights_list)
        self.loss = tf.reduce_mean(seq_loss)
    
    def build_optimizer(self):
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.minimize(self.loss)

    def partial_fit(self, x, y, measure_only=False):
        x_input, x_weight = zip(*x)
        feed_dict = {
            self.x_input:x_input,
            self.x_weight:x_weight,
            self.y_input:y,
            self.dropout_keep:1.-self.dropout
        }
        
        if measure_only:
            loss = self.session.run(self.loss, feed_dict)
        else:
            loss, _ = self.session.run([self.loss, self.train_op], feed_dict)
        return loss
    
    def predict(self, x):
        x_input, x_weight = zip(*x)
        feed_dict = {
            self.x_input:x_input,
            self.x_weight:x_weight,
            self.dropout_keep:1.
        }
        predictions = self.session.run(self.labels, feed_dict)
        return predictions

In [19]:
tf.reset_default_graph()
session = tf.InteractiveSession()

params = {
    'learning_rate':.001,
    'xvocab':xvocab,
    'yvocab':yvocab,
    'max_time':max_time,
    'max_chargrams':max_chargrams,
    'char_embed_size':50,
    'hidden_size':100,
    'dropout':.0
}
ner = ChargramNER(session, **params)

Model Loading... 41 (?, 5)
Done


In [20]:
ner.fit(zip(train_xs, train_ws), train_ys, batch_size=64, n_epoch=5)


 Epoch 1/5 : Batch 38/38: Loss 0.3032
 Epoch 2/5 : Batch 38/38: Loss 0.2519
 Epoch 3/5 : Batch 38/38: Loss 0.2043
 Epoch 4/5 : Batch 38/38: Loss 0.1395
 Epoch 5/5 : Batch 38/38: Loss 0.0608


In [20]:
preds = ner.predict(zip(test_xs, test_ws))
preds = preds.tolist()

In [21]:
target_ys = depad_sequences(test_ys, yvocab.ipad)
predictions = depad_sequences(preds, test_ws)

In [22]:
flat_targets = [ yvocab.token(t) for y in target_ys for t in y ]
flat_predictions = [ yvocab.token(p) for y in predictions for p in y ]
stats = macro_f1(flat_targets, flat_predictions)

In [23]:
print stats

{'macro_recall': 0.33333333333333331, 'macro_f1': 0.32135652247775565, 'macro_precision': 0.31021052415800587, 'scores': {'I': {'recall': 0.0, 'support': 467, 'precision': 0.0, 'f1': 0.0}, 'B': {'recall': 0.0, 'support': 661, 'precision': 0.0, 'f1': 0.0}, 'O': {'recall': 1.0, 'support': 15133, 'precision': 0.9306315724740176, 'f1': 0.964069567433267}}}
